# EE123 Final Project Sp20 - Video Communication
### Bryan Yang and Lixing He

In [35]:
# Import functions and libraries (Part I)
import numpy as np
import scipy.fftpack
from scipy.fftpack import dct, idct
import matplotlib.pyplot as plt
from bitarray import bitarray
from huffman import *
from os import stat
from PIL import Image


# Import functions and libraries (Part II)
import queue as Queue
import threading,time
import sys

from numpy import pi
from numpy import sin
from numpy import zeros
from numpy import r_
from numpy import ones
from scipy import signal
from scipy import integrate

import time
import multiprocessing

from numpy import mean
from numpy import power
from numpy.fft import fft
from numpy.fft import fftshift
from numpy.fft import ifft
from numpy.fft import ifftshift
from  scipy.io.wavfile import read as wavread
#import ax25
from math import gcd
import sounddevice as sd
import RPi.GPIO as GPIO
from functools import reduce
from numpy import ones,zeros, pi, cos, exp, sign


# Import functions and libraries (Part III)
import base64
import threading
import newax25 as ax25
import bitarray as barr

from scipy import misc
import Helper_functions as hfunc
import os

%matplotlib inline

###### Commands to run

Compile PLL

In [2]:
!cc -fPIC -shared -o libpll.so pll.c

In [3]:
import numpy.ctypeslib as npct
from ctypes import c_int
from ctypes import c_float

array_1d_int = npct.ndpointer(dtype=np.int, ndim=1, flags='CONTIGUOUS')

libcd = npct.load_library("./libpll", ".")
libcd.pll.restype = c_int
libcd.pll.argtypes= [array_1d_int, c_int, array_1d_int,array_1d_int,  array_1d_int,array_1d_int, c_int, c_float]

Set up loopback

In [4]:
!sudo modprobe snd_aloop
!aplay -L | grep -i plughw:card=loopback
!aplay -l | grep -i loopback

plughw:CARD=Loopback,DEV=0
plughw:CARD=Loopback,DEV=1
card 2: Loopback [Loopback], device 0: Loopback PCM [Loopback PCM]
card 2: Loopback [Loopback], device 1: Loopback PCM [Loopback PCM]


In [5]:
sd.query_devices()

  0 bcm2835 ALSA: - (hw:0,0), ALSA (0 in, 2 out)
  1 bcm2835 ALSA: IEC958/HDMI (hw:0,1), ALSA (0 in, 2 out)
  2 Fe-Pi Audio: - (hw:1,0), ALSA (2 in, 2 out)
  3 Loopback: PCM (hw:2,0), ALSA (32 in, 32 out)
  4 Loopback: PCM (hw:2,1), ALSA (32 in, 32 out)
  5 sysdefault, ALSA (0 in, 128 out)
  6 dmix, ALSA (0 in, 2 out)
* 7 default, ALSA (2 in, 2 out)

In [6]:
builtin_idx = 0
usb_idx = 2
loop_idx = 4  # Use loopback dev 1! direwolf is connected to dev 0.
loop_idx_rcv = 3 # Use loopback dev 0. 
sd.default.samplerate=48000
sd.default.channels = 1

Set up direwolf

In [ ]:
# Enter your address here
myaddress = "Cory Hall Berkeley CA USA"

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="EE123")
location = geolocator.geocode(myaddress)

In [ ]:
# hopefully it did not fail
print(location.address)
print((location.latitude, location.longitude))

In [ ]:
import math
lat = "lat=%d^%2.2f%s" % (int(abs(location.latitude)),
                          60*(abs(location.latitude) - abs(math.floor(location.latitude))),
                          ("N") if location.latitude >0 else "S" )  
long = "long=%d^%2.2f%s" % (int(abs(location.longitude)),
                            60*(abs(location.longitude) - math.floor(abs(location.longitude))),
                          ("W") if location.longitude <0 else "E" )  
                            
print(lat, long)

In [ ]:
# enter your callsign and comment for the beacon

callsign = "KN6IDV-1"
comment = "EE123 is cool"


callsignr = callsign.split('-')[0].upper()
code = 0x73e2
for i, char in enumerate(callsignr):
    code ^= ord(char) << (8 if not i % 2 else 0)

passcode = code&0x7fff
print("Passcode:", passcode)

In [ ]:
import os

cmd = "cat direwolf-loopback-DRAFT.conf  | sed  's/EE123_CALL/"+callsign+"/g'  | "
cmd = cmd +  "sed  's/EE123_PASSCODE/%d/g' | " % (passcode)
cmd = cmd +  "sed  's/EE123_COMMENT/comment=\"%s\"/g' | " % (comment)
cmd = cmd +  "sed  's/EE123_LATCOORD/%s/g' | " % (lat)
cmd = cmd +  "sed  's/EE123_LONGCOORD/%s/g' > direwolf-loopback.conf" % (long)
print(cmd)
os.system(cmd)  # NOTE: this is poor security practice, but conveniently concise here

### Functions from Part I

Color Conversion

In [7]:
con = np.array([-128,0,0])
scl = np.array([[0.299,0.587,0.114],[-0.168736,-0.331264,0.5],[0.5,-0.418688,-0.081312]])
def RGB2YCbCr(im_rgb):
    # Input:  a 3D float array, im_rgb, representing an RGB image in range [0.0,255.0]
    # Output: a 3D float array, im_ycbcr, representing a YCbCr image in range [-128.0,127.0]
    
    # Your code here
    im_ycbcr = np.zeros(im_rgb.shape)
    for r in range(im_rgb.shape[0]):
        for c in range(im_rgb.shape[1]):
            ycbcr = con+scl.dot(im_rgb[r][c])
            for i in range(ycbcr.shape[0]):
                ycbcr[i] = clip(ycbcr[i],-128.0,127.0)
            im_ycbcr[r][c] = ycbcr
    return im_ycbcr

def YCbCr2RGB(im_ycbcr):
    # Input:  a 3D float array, im_ycbcr, representing a YCbCr image in range [-128.0,127.0]
    # Output: a 3D float array, im_rgb, representing an RGB image in range [0.0,255.0]
    
    # Your code here
    im_rgb = np.zeros(im_ycbcr.shape)
    for r in range(im_ycbcr.shape[0]):
        for c in range(im_ycbcr.shape[1]):
            rgb = np.linalg.inv(scl).dot(im_ycbcr[r][c] - con)
            for i in range(rgb.shape[0]):
                rgb[i] = clip(rgb[i],0.0,255.0)
            im_rgb[r][c] = rgb
    return im_rgb

def clip(num,low,high):
    return low if num < low else (high if num > high else num)


def chroma_downsample(C):
    # Input:  an MxN array, C, of chroma values
    # Output: an (M/2)x(N/2) array, C2, of downsampled chroma values
    
    # Your code here
    C2 = np.array(Image.fromarray(C).resize((C.shape[1]//2, C.shape[0]//2), resample=Image.BILINEAR))
    return C2

def chroma_upsample(C2):
    # Input:  an (M/2)x(N/2) array, C2, of downsampled chroma values
    # Output: an MxN array, C, of chroma values
    
    # Your code here
    C = np.array(Image.fromarray(C2).resize((C2.shape[1]*2, C2.shape[0]*2), resample=Image.BILINEAR))
    return C

DCT

In [8]:
def dct2(block):
    # Input:  a 2D array, block, representing an image block
    # Output: a 2D array, block_c, of DCT coefficients
    
    # Your code here
    block_c = scipy.fftpack.dct(block, axis=1, norm='ortho')
    block_c = scipy.fftpack.dct(block_c, axis=0, norm='ortho')
                
    return block_c

def idct2(block_c):
    # Input:  a 2D array, block_c, of DCT coefficients
    # Output: a 2D array, block, representing an image block
    
    # Your code here
    block = scipy.fftpack.idct(block_c, axis=1, norm='ortho')
    block = scipy.fftpack.idct(block, axis=0, norm='ortho')
    
    return block

Quantization

In [9]:
def quantize(block_c, mode="y", quality=75):
    # Input:  a 2D float array, block_c, of DCT coefficients
    #         a string, mode, ("y" for luma quantization, "c" for chroma quantization)
    #         an int, quality, the JPEG quality factor in range [1,100] (defaults to 75)
    # Output: a 2D int array, block_cq, of quantized DCT coefficients
    
    if mode is "y":
        Q = np.array([[ 16,  11,  10,  16,  24,  40,  51,  61 ],
                      [ 12,  12,  14,  19,  26,  58,  60,  55 ],
                      [ 14,  13,  16,  24,  40,  57,  69,  56 ],
                      [ 14,  17,  22,  29,  51,  87,  80,  62 ],
                      [ 18,  22,  37,  56,  68,  109, 103, 77 ],
                      [ 24,  36,  55,  64,  81,  104, 113, 92 ],
                      [ 49,  64,  78,  87,  103, 121, 120, 101],
                      [ 72,  92,  95,  98,  112, 100, 103, 99 ]])
    elif mode is "c":
        Q = np.array([[ 17,  18,  24,  47,  99,  99,  99,  99 ],
                      [ 18,  21,  26,  66,  99,  99,  99,  99 ],
                      [ 24,  26,  56,  99,  99,  99,  99,  99 ],
                      [ 47,  66,  99,  99,  99,  99,  99,  99 ],
                      [ 99,  99,  99,  99,  99,  99,  99,  99 ],
                      [ 99,  99,  99,  99,  99,  99,  99,  99 ],
                      [ 99,  99,  99,  99,  99,  99,  99,  99 ],
                      [ 99,  99,  99,  99,  99,  99,  99,  99 ]]) 
    else:
        raise Exception("String argument must be 'y' or 'c'.")
        
    if quality < 1 or quality > 100:
        raise Exception("Quality factor must be in range [1,100].")
    
    scalar = 5000 / quality if quality < 50 else 200 - 2 * quality # formula for scaling by quality factor
    Q = Q * scalar / 100. # scale the quantization matrix
    Q[Q<1.] = 1. # do not divide by numbers less than 1
    
    # Your code here
    block_cq = np.zeros(block_c.shape)
    for r in range(block_c.shape[0]):
        for c in range(block_c.shape[1]):
            block_cq[r][c] = int(np.round(block_c[r][c]/Q[r][c]))
    
    return block_cq.astype(int)

def unquantize(block_cq, mode="y", quality=75):
    # Input:  a 2D int array, block_cq, of quantized DCT coefficients
    #         a string, mode, ("y" for luma quantization, "c" for chroma quantization)
    #         an int, quality, the JPEG quality factor in range [1,100] (defaults to 75)
    # Output: a 2D float array, block_c, of "unquantized" DCT coefficients (they will still be quantized)
    
    if mode is "y":
        Q = np.array([[ 16,  11,  10,  16,  24,  40,  51,  61 ],
                      [ 12,  12,  14,  19,  26,  58,  60,  55 ],
                      [ 14,  13,  16,  24,  40,  57,  69,  56 ],
                      [ 14,  17,  22,  29,  51,  87,  80,  62 ],
                      [ 18,  22,  37,  56,  68,  109, 103, 77 ],
                      [ 24,  36,  55,  64,  81,  104, 113, 92 ],
                      [ 49,  64,  78,  87,  103, 121, 120, 101],
                      [ 72,  92,  95,  98,  112, 100, 103, 99 ]])
    elif mode is "c":
        Q = np.array([[ 17,  18,  24,  47,  99,  99,  99,  99 ],
                      [ 18,  21,  26,  66,  99,  99,  99,  99 ],
                      [ 24,  26,  56,  99,  99,  99,  99,  99 ],
                      [ 47,  66,  99,  99,  99,  99,  99,  99 ],
                      [ 99,  99,  99,  99,  99,  99,  99,  99 ],
                      [ 99,  99,  99,  99,  99,  99,  99,  99 ],
                      [ 99,  99,  99,  99,  99,  99,  99,  99 ],
                      [ 99,  99,  99,  99,  99,  99,  99,  99 ]]) 
    else:
        raise Exception("String argument must be 'y' or 'c'.")
            
    if quality < 1 or quality > 100:
        raise Exception("Quality factor must be in range [1,100].")
        
    scalar = 5000 / quality if quality < 50 else 200 - 2 * quality # formula for scaling by quality factor
    Q = Q * scalar / 100. # scale the quantization matrix
    Q[Q<1.] = 1. # do not divide by numbers less than 1
    
    # Your code here
    block_c = np.zeros(np.array(block_cq).shape)
    for r in range(block_c.shape[0]):
        for c in range(block_c.shape[1]):
            block_c[r][c] = np.round(block_cq[r][c]*Q[r][c])
    
    return block_c  

Run-length Encoding

In [10]:
def zigzag(block_cq):
    # Input:  a 2D array, block_cq, of quantized DCT coefficients
    # Output: a list, block_cqz, of zig-zag reordered quantized DCT coefficients
    
    idx = [0, 1, 8, 16, 9, 2, 3, 10, 17, 24, 32, 25, 18, 11, 4, 5, 12, 19, 26, 33, 40, 48, 41,
           34, 27, 20, 13, 6, 7, 14, 21, 28, 35, 42, 49, 56, 57, 50, 43, 36, 29, 22, 15, 23,
           30, 37, 44, 51, 58, 59, 52, 45, 38, 31, 39, 46, 53, 60, 61, 54, 47, 55, 62, 63]
    
    # Your code here
    block_cqz = np.ravel(block_cq, order='C')[idx]
    
    return block_cqz.tolist()

def unzigzag(block_cqz):
    # Input:  a list, block_cqz, of zig-zag reordered quantized DCT coefficients
    # Output: a 2D array, block_cq, of conventionally ordered quantized DCT coefficients
    
    idx = [0, 1, 5, 6, 14, 15, 27, 28, 2, 4, 7, 13, 16, 26, 29, 42, 3, 8, 12, 17, 25, 30, 41, 
           43, 9, 11, 18, 24, 31, 40, 44, 53, 10, 19, 23, 32, 39, 45, 52, 54, 20, 22, 33, 38,
           46, 51, 55, 60, 21, 34, 37, 47, 50, 56, 59, 61, 35, 36, 48, 49, 57, 58, 62, 63]
    
    # Your code here
    block_cq = np.array(block_cqz)[idx]
    block_cq = np.reshape(block_cq,(8,8),order='C')
    
    return block_cq.tolist()


def zrle(block_cqz):
    # Input:  a list, block_cqz, of zig-zag reordered quantized DCT coefficients
    # Output: a list, block_cqzr, of zero-run-length encoded quantized DCT coefficients
    
    # Your code here
    block_cqzr = [block_cqz[0]]
    zero_ctr = 0
    for i in range(1,len(block_cqz)):
        if block_cqz[i] == 0:
            zero_ctr += 1
            if zero_ctr == 16 and any(n != 0 for n in block_cqz[i:]):
                block_cqzr.append((15,0))
                zero_ctr = 0
                #print((15,0))
        else:
            block_cqzr.append((zero_ctr, block_cqz[i]))
            #print((zero_ctr, block_cqz[i]))
            zero_ctr = 0
            
    block_cqzr.append((0,0))   
    
    return block_cqzr

def unzrle(block_cqzr):
    # Input:  a list, block_cqzr, of zero-run-length encoded quantized DCT coefficients
    # Output: a list, block_cqz, of zig-zag reordered quantized DCT coefficients
    
    # Your code here
    block_cqz = [block_cqzr[0]]
    for zeros, value in block_cqzr[1:]:
        if zeros == 0 and value == 0:
            #should be last item in loop
            break
        block_cqz.extend([0]*zeros)
        block_cqz.append(value)
    if len(block_cqz) < 64:
        block_cqz.extend([0]*(64-len(block_cqz)))
    return block_cqz

Huffman Encoding

In [11]:
def encode_block(block, mode="y", quality=75):
    # Input:  a 2D array, block, representing an image component block
    #         a string, mode, ("y" for luma, "c" for chroma)
    #         an int, quality, the JPEG quality factor in range [1,100] (defaults to 75)
    # Output: a bitarray, dc_bits, of Huffman encoded DC coefficients
    #         a bitarray, ac_bits, of Huffman encoded AC coefficients
    
    block_c = dct2(block)
    block_cq = quantize(block_c, mode, quality)
    block_cqz = zigzag(block_cq)
    block_cqzr = zrle(block_cqz)
    dc_bits = encode_huffman(block_cqzr[0], mode) # DC
    ac_bits = ''.join(encode_huffman(v, mode) for v in block_cqzr[1:]) # AC
    
    return bitarray(dc_bits), bitarray(ac_bits)

def decode_block(dc_gen, ac_gen, mode="y", quality=75):
    # Inputs: a generator, dc_gen, that yields decoded Huffman DC coefficients
    #         a generator, ac_gen, that yields decoded Huffman AC coefficients
    #         a string, mode, ("y" for luma, "c" for chroma)
    #         an int, quality, the JPEG quality factor in range [1,100] (defaults to 75)
    # Output: a 2D array, block, decoded by and yielded from the two generators
    
    block_cqzr = [next(dc_gen)] # initialize list by yielding from DC generator
    while block_cqzr[-1] != (0,0):
        block_cqzr.append(next(ac_gen)) # append to list by yielding from AC generator until (0,0) is encountered
    block_cqz = unzrle(block_cqzr)
    block_cq = unzigzag(block_cqz)
    block_c = unquantize(block_cq, mode, quality)
    block = idct2(block_c)
    
    return block

Compression

In [12]:
def mirror_pad(img):
    # Input:  a 3D float array, img, representing an RGB image in range [0.0,255.0]
    # Output: a 3D float array, img_pad, mirror padded so the number of rows and columns are multiples of 16
    
    M, N = img.shape[0:2]
    pad_r = ((16 - (M % 16)) % 16) # number of rows to pad
    pad_c = ((16 - (N % 16)) % 16) # number of columns to pad
    img_pad = np.pad(img, ((0,pad_r), (0,pad_c), (0,0)), "symmetric") # symmetric padding
    
    return img_pad

def encode_image(img, quality=75):
    # Inputs:  a 3D float array, img, representing an RGB image in range [0.0,255.0]
    #          an int, quality, the JPEG quality factor in range [1,100] (defaults to 75)
    # Outputs: a bitarray, Y_dc_bits, the Y component DC bitstream
    #          a bitarray, Y_ac_bits, the Y component AC bitstream
    #          a bitarray, Cb_dc_bits, the Cb component DC bitstream
    #          a bitarray, Cb_ac_bits, the Cb component AC bitstream
    #          a bitarray, Cr_dc_bits, the Cr component DC bitstream
    #          a bitarray, Cr_ac_bits, the Cr component AC bitstream
    
    M_orig, N_orig = img.shape[0:2]
    img = mirror_pad(img[:,:,0:3])
    M, N = img.shape[0:2]
    
    im_ycbcr = RGB2YCbCr(img)    
    Y = im_ycbcr[:,:,0]
    Cb = chroma_downsample(im_ycbcr[:,:,1])
    Cr = chroma_downsample(im_ycbcr[:,:,2])
    
    # Y component
    Y_dc_bits = bitarray()
    Y_ac_bits = bitarray()
    for i in np.r_[0:M:8]:
        for j in np.r_[0:N:8]:
            block = Y[i:i+8,j:j+8]
            dc_bits, ac_bits = encode_block(block, "y", quality)
            Y_dc_bits.extend(dc_bits)
            Y_ac_bits.extend(ac_bits)
    
    # Cb component
    Cb_dc_bits = bitarray()
    Cb_ac_bits = bitarray()
    for i in np.r_[0:M//2:8]:
        for j in np.r_[0:N//2:8]:
            block = Cb[i:i+8,j:j+8]
            dc_bits, ac_bits = encode_block(block, "c", quality)
            Cb_dc_bits.extend(dc_bits)
            Cb_ac_bits.extend(ac_bits)
            
    # Cr component
    Cr_dc_bits = bitarray()
    Cr_ac_bits = bitarray()
    for i in np.r_[0:M//2:8]:
        for j in np.r_[0:N//2:8]:
            block = Cr[i:i+8,j:j+8]
            dc_bits, ac_bits = encode_block(block, "c", quality)
            Cr_dc_bits.extend(dc_bits)
            Cr_ac_bits.extend(ac_bits)
            
    bits = (Y_dc_bits, Y_ac_bits, Cb_dc_bits, Cb_ac_bits, Cr_dc_bits, Cr_ac_bits)
    
    return bits

def decode_image(bits, M, N, quality=75):
    # Inputs: a tuple, bits, containing the following:
    #              a bitarray, Y_dc_bits, the Y component DC bitstream
    #              a bitarray, Y_ac_bits, the Y component AC bitstream
    #              a bitarray, Cb_dc_bits, the Cb component DC bitstream
    #              a bitarray, Cb_ac_bits, the Cb component AC bitstream
    #              a bitarray, Cr_dc_bits, the Cr component DC bitstream
    #              a bitarray, Cr_ac_bits, the Cr component AC bitstream
    #         ints, M and N, the number of rows and columns in the image
    #         an int, quality, the JPEG quality factor in range [1,100] (defaults to 75)
    # Output: a 3D float array, img, representing an RGB image in range [0.0,255.0]
    
    Y_dc_bits, Y_ac_bits, Cb_dc_bits, Cb_ac_bits, Cr_dc_bits, Cr_ac_bits = bits # unpack bits tuple
    
    M_orig = M # save original image dimensions
    N_orig = N
    M = M_orig + ((16 - (M_orig % 16)) % 16) # dimensions of padded image
    N = N_orig + ((16 - (N_orig % 16)) % 16)
    num_blocks = M * N // 64 # number of blocks
    
    # Y component
    Y_dc_gen = decode_huffman(Y_dc_bits.to01(), "dc", "y")
    Y_ac_gen = decode_huffman(Y_ac_bits.to01(), "ac", "y")
    Y = np.empty((M, N))
    for b in range(num_blocks):
        block = decode_block(Y_dc_gen, Y_ac_gen, "y", quality)
        r = (b*8 // N)*8 # row index (top left corner)
        c = b*8 % N # column index (top left corner)
        Y[r:r+8, c:c+8] = block

    # Cb component
    Cb_dc_gen = decode_huffman(Cb_dc_bits.to01(), "dc", "c")
    Cb_ac_gen = decode_huffman(Cb_ac_bits.to01(), "ac", "c")
    Cb2 = np.empty((M//2, N//2))
    for b in range(num_blocks//4):
        block = decode_block(Cb_dc_gen, Cb_ac_gen, "c", quality)
        r = (b*8 // (N//2))*8 # row index (top left corner)
        c = b*8 % (N//2) # column index (top left corner)
        Cb2[r:r+8, c:c+8] = block

    # Cr component
    Cr_dc_gen = decode_huffman(Cr_dc_bits.to01(), "dc", "c")
    Cr_ac_gen = decode_huffman(Cr_ac_bits.to01(), "ac", "c")
    Cr2 = np.empty((M//2, N//2))
    for b in range(num_blocks//4):
        block = decode_block(Cr_dc_gen, Cr_ac_gen, "c", quality)
        r = (b*8 // (N//2))*8 # row index (top left corner)
        c = b*8 % (N//2) # column index (top left corner)
        Cr2[r:r+8, c:c+8] = block

    Cb = chroma_upsample(Cb2)
    Cr = chroma_upsample(Cr2)

    img = YCbCr2RGB(np.stack((Y,Cb,Cr), axis=-1))
    
    img = img[0:M_orig,0:N_orig,:] # crop out padded parts

    return img

JPEG123

In [13]:
def jpeg123_encoder(img, outfile, quality=75):
    # Inputs: a 3D uint8 array, img, representing an RGB color image
    #         a string, outfile, of the output binary filename
    #         an int, quality, the JPEG quality factor in range [1,100] (defaults to 75)
    
    img = img.astype(np.float64)
    M, N = img.shape[0:2]
    
    bits = encode_image(img, quality=quality)

    with open(outfile, "wb") as fh:
        # Your code here        
        SOI = bitarray(endian='big')
        SOI.frombytes(bytes.fromhex("FFD8"))
        SOS = bitarray(endian='big')
        SOS.frombytes(bytes.fromhex("FFDA"))
        EOI = bitarray(endian='big')
        EOI.frombytes(bytes.fromhex("FFD9"))
        
        #bitarray(bytes, endian='big') big by default
        SOI.tofile(fh)
        #rows
        rows = bitarray(endian='big')
        rows.frombytes(M.to_bytes(2, 'big'))
        rows.tofile(fh)
        #columns
        cols = bitarray(endian='big')
        cols.frombytes(N.to_bytes(2, 'big'))
        cols.tofile(fh)
        #quality factor
        q_fact = bitarray(endian='big')
        q_fact.frombytes(quality.to_bytes(2, 'big'))
        q_fact.tofile(fh)
        #in bits: index 0 = Y DC, 1 = Y AC, 2 = Cb DC, 3 = Cb AC, 4 = Cr DC, 5 = Cr AC
        for bitarr in bits:
            SOS.tofile(fh)
            bitarr.extend(bitarray([1]*((16 - (len(bitarr) % 16)) % 16), endian='big'))
            bitarr.tofile(fh)
        EOI.tofile(fh)
        
def jpeg123_decoder(infile):
    # Inputs:  a string, infile, of the input binary filename
    # Outputs: a 3D uint8 array, img_dec, representing a decoded JPEG123 color image
    
    with open(infile, "rb") as fh:
        SOI = fh.read(2)
        if SOI != bytes.fromhex("FFD8"):
            raise Exception("Start of Image marker not found!")
        M = int.from_bytes(fh.read(2), "big")
        N = int.from_bytes(fh.read(2), "big")
        quality = int.from_bytes(fh.read(2), "big")
        SOS = fh.read(2)
        if SOS != bytes.fromhex("FFDA"):
            raise Exception("Start of Scan marker not found!")
        bits = ()
        for _ in range(5):
            ba = bitarray()
            for b in iter(lambda: fh.read(2), bytes.fromhex("FFDA")): # iterate until next SOS marker
                ba.frombytes(b)
            bits = (*bits, ba)
        ba = bitarray()
        for b in iter(lambda: fh.read(2), bytes.fromhex("FFD9")): # iterate until EOI marker
            ba.frombytes(b)
        bits = (*bits, ba)
    
    img_dec = decode_image(bits, M, N, quality)
    
    return img_dec.astype(np.uint8)

### Functions from Part II

Plotting stuff

In [14]:
# function to compute least common multiple
def lcm(numbers):
    return reduce(lambda x, y: (x*y)//gcd(x,y), numbers, 1)

# function to compute average power spectrum
def avgPS( x, N=256, fs=1):
    M = (len(x)//N)
    x_ = np.reshape(x[:M*N],(M,N)) * np.hamming(N)[None,:]
    X = np.fft.fftshift(np.fft.fft(x_,axis=1),axes=1)
    return r_[-N/2.0:N/2.0]/N*fs, np.mean(abs(X)**2,axis=0)



# Plot an image of the spectrogram y, with the axis labeled with time tl,
# and frequency fl
#
# t_range -- time axis label, nt samples
# f_range -- frequency axis label, nf samples
# y -- spectrogram, nf by nt array
# dbf -- Dynamic range of the spect

def sg_plot( t_range, f_range, y, dbf = 60, fig = None) :
    eps = 10.0**(-dbf/20.0)  # minimum signal
    
    # find maximum
    y_max = abs(y).max()
    
    # compute 20*log magnitude, scaled to the max
    y_log = 20.0 * np.log10( (abs( y ) / y_max)*(1-eps) + eps )
    
    # rescale image intensity to 256
    img = 256*(y_log + dbf)/dbf - 1
    
    fig=plt.figure(figsize=(16,6))
    
    plt.imshow( np.flipud( 64.0*(y_log + dbf)/dbf ), extent= t_range  + f_range ,cmap=plt.cm.gray, aspect='auto')
    plt.xlabel('Time, s')
    plt.ylabel('Frequency, Hz')
    plt.tight_layout()
    
    return fig


def myspectrogram_hann_ovlp(x, m, fs, fc,dbf = 60):
    # Plot the spectrogram of x.
    # First take the original signal x and split it into blocks of length m
    # This corresponds to using a rectangular window %
    
    
    isreal_bool = np.isreal(x).all()
    
    # pad x up to a multiple of m 
    lx = len(x);
    nt = (lx + m - 1) // m
    x = np.append(x,zeros(-lx+nt*m))
    x = x.reshape((m//2,nt*2), order='F')
    x = np.concatenate((x,x),axis=0)
    x = x.reshape((m*nt*2,1),order='F')
    x = x[r_[m//2:len(x),np.ones(m//2)*(len(x)-1)].astype(int)].reshape((m,nt*2),order='F')
    
    
    xmw = x * np.hanning(m)[:,None];
    
    
    # frequency index
    t_range = [0.0, lx / fs]
    
    if isreal_bool:
        f_range = [ fc, fs / 2.0 + fc]
        xmf = np.fft.fft(xmw,len(xmw),axis=0)
        sg_plot(t_range, f_range, xmf[0:m//2,:],dbf=dbf)
        print (1)
    else:
        f_range = [-fs / 2.0 + fc, fs / 2.0 + fc]
        xmf = np.fft.fftshift( np.fft.fft( xmw ,len(xmw),axis=0), axes=0 )
        sg_plot(t_range, f_range, xmf,dbf = dbf)
    
    return t_range, f_range, xmf

from Lab 4 Part II

In [ ]:
def afsk1200(bits, fs = 48000):
    # the function will take a bitarray of bits and will output an AFSK1200 modulated signal of them, sampled at fs Hz
    #  Inputs:
    #         bits  - bitarray of bits
    #         fs    - sampling rate
    # Outputs:
    #         sig    -  returns afsk1200 modulated signal samples at fs Hz
    
    #from provided solution to Lab 4 Part I (it ain't part II)

    fss = lcm((1200,fs))
    deci = fss//fs

    Nb = fss//1200
    nb = len(bits)
    NRZ = ones((nb,Nb))
    for n in range(0,nb):
        if bits[n]:
            NRZ[n,:]=-NRZ[n,:]

    freq = 1700 + 500*NRZ.ravel()
    ph = 2.0*pi*integrate.cumtrapz(freq)/fss
    sig = cos(ph[::deci])
    
    
    return sig






def nc_afsk1200Demod(sig, fs=48000.0, TBW=2.0):
    #  non-coherent demodulation of afsk1200
    # function returns the NRZ (without rectifying it)
    # 
    # sig  - signal
    # baud - The bitrate. Default 1200
    # fs   - sampling rate in Hz
    # TBW  - TBW product of the filters
    #
    # Returns:
    #     NRZ 
    
    N = (int(fs/1200*TBW)//2)*2+1
    
    BW = TBW/(1.0*N/fs)
    BW = TBW/(1.0*N/fs) #why is this here twice?
    h_lp = signal.firwin(N,BW/fs*1.0,window='hanning')
    h_lpp = signal.firwin(N,BW*2*1.2/fs,window='hanning')
    h_space = h_lp*exp(1j*2*pi*(2200)*r_[-N/2:N/2]/fs)
    h_mark = h_lp*exp(1j*2*pi*(1200)*r_[-N/2:N/2]/fs)
    h_bp = (signal.firwin(N,BW/fs*2.2,window='hanning'))*exp(1j*2*pi*1700*r_[-N/2:N/2]/fs)

    sig = np.convolve(sig.copy(),h_bp,'same')
    mark = abs(np.convolve(sig,h_mark,mode='same'))
    space = abs(np.convolve(sig,h_space,mode='same'))
    NRZ = mark-space
    NRZ = np.convolve(NRZ,h_lpp,'same')

    return NRZ


def PLL(NRZa, a = 0.74 , fs = 48000, baud = 1200):
    
    Ns = 1.0*fs/baud
    idx = zeros(len(NRZa)//int(Ns)*2)   # allocate space to save indexes
    lock = zeros(len(NRZa)//int(Ns)*2)
    c = 0
    dpll = np.round(2**32 / Ns).astype(np.int32)
    pll = np.int32(0)
    ppll = pll - dpll
    
    for n in range(1,len(NRZa)):
        if (pll < 0) and (ppll >0):
            idx[c] = n
            c = c+1
    
        if (NRZa[n] >= 0) !=  (NRZa[n-1] >=0):
            pll = np.int32(pll*a)
            lock[c] = pll
    
        ppll = pll
        pll = np.int32(pll+ dpll)
    
    return idx[:c].astype(np.int32) 
    

most stuff from final part 2 covered in modem class from final part 3, so ^ that stuff isn't necessary

### Functions from Part III

Modem class

In [36]:
class TNCaprs: 
    def __init__(self, fs = 48000.0, Abuffer = 1024, Nchunks=10):
        
        #  Implementation of an afsk1200 TNC. 
        #
        #  The TNC processes a `Abuffer` long buffers, till `Nchunks` number of buffers are 
        #  collected into a large one.
        #  This is because python is able to more efficiently process larger buffers than smaller ones.
        #  Then, the resulting large buffer is demodulated, sampled and packets extracted.
        #
        # Inputs:
        #    fs  - sampling rate
        #   TBW  -  TBW of the demodulator filters
        #   Abuffer - Input audio buffers from Pyaudio
        #   Nchunks - Number of audio buffers to collect before processing
        #   plla    - agressivness parameter of the PLL
        
        ## compute sizes based on inputs
        self.TBW = 2.0   # TBW for the demod filters
        self.N = (int(fs/1200*self.TBW)//2)*2+1   # length of the mark-space filters for demod
        self.fs = fs     # sampling rate   
        self.BW = 1200      # BW of filter based on TBW
        self.Abuffer = Abuffer             # size of audio buffer
        self.Nchunks = Nchunks             # number of audio buffers to collect
        self.Nbuffer = Abuffer*Nchunks+(self.N*3-3)         # length of the large buffer for processing
        self.Ns = 1.0*fs/1200.0 # samples per symbol
        
        ## state variables for the modulator
        self.prev_ph = 0  # previous phase to maintain continuous phase when recalling the function
        
        ##  Generate Filters for the demodulator    
        filt_BW = self.TBW/(1.0*self.N/fs)
        self.h_lp = signal.firwin(self.N,filt_BW/fs*1.0,window='hanning')
        self.h_lpp = signal.firwin(self.N,filt_BW*2*1.2/fs,window='hanning')
        self.h_space = self.h_lp*exp(1j*2*pi*(2200)*r_[-self.N/2:self.N/2]/fs)
        self.h_mark = self.h_lp*exp(1j*2*pi*(1200)*r_[-self.N/2:self.N/2]/fs)
        self.h_bp = (signal.firwin(self.N,filt_BW/fs*2.2,window='hanning'))*exp(1j*2*pi*1700*r_[-self.N/2:self.N/2]/fs)

        ## PLL state variables  -- so conntinuity between buffers is preserved
        self.dpll = np.round(2.0**32 / self.Ns).astype(np.int32)    # PLL step
        self.pll =  0                # PLL counter
        self.ppll = -self.dpll       # PLL counter previous value -- to detect overflow
        self.plla = 0.74             # PLL agressivness (small more agressive)
    
        ## state variable to NRZI2NRZ
        self.NRZIprevBit = bool(1)  
        
        ## State variables for findPackets
        self.state='search'   # state variable:  'search' or 'pkt'
        self.pktcounter = 0   # counts the length of a packet
        self.packet = barr.bitarray([0,1,1,1,1,1,1,0])   # current packet being collected
        self.bitpointer = 0   # poiter to advance the search beyond what was already 
                              # searched in the previous buffer

        ## State variables for processBuffer
        self.buff = zeros(self.Nbuffer)   # large overlapp-save buffer
        self.chunk_count = 0              # chunk counter
        # bits from end of prev buffer to be copied to beginning of new
        self.oldbits = barr.bitarray([0,0,0,0,0,0,0])    
        self.Npackets = 0                 # packet counter
        
    
    def NRZ2NRZI(self,NRZ, prevBit = True):
        NRZI = NRZ.copy() 
        for n in range(0,len(NRZ)):
            if NRZ[n] :
                NRZI[n] = prevBit
            else:
                NRZI[n] = not(prevBit)
            prevBit = NRZI[n]
        return NRZI
    

    def NRZI2NRZ(self, NRZI):  
        NRZ = NRZI.copy() 
    
        for n in range(0,len(NRZI)):
            NRZ[n] = NRZI[n] == self.NRZIprevBit
            self.NRZIprevBit = NRZI[n]
    
        return NRZ
    
    
    def KISS2bits(self,KISS):
        # function that takes a KISS frame sent via TCP/IP and converts it to an APRSpacket bit stream.
        
        bits = barr.bitarray(endian="little")
        bits.frombytes(KISS)
        fcs = ax25.FCS()
        for bit in bits:
            fcs.update_bit(bit)
            
        bits.frombytes(fcs.digest())
        return barr.bitarray('01111110') + ax25.bit_stuff(bits) + barr.bitarray('01111110') 
     
        
    def bits2KISS(self,bits):
        # function that takes a bitstream of an APRS-packet, removes flags and FCS and unstuffs the bits
        bitsu = ax25.bit_unstuff(bits[8:-8])
        return  bitsu[:-16].tobytes() 
    
    
    def modulate(self,bits):
    # the function will take a bitarray of bits and will output an AFSK1200 modulated signal of them, 
    # sampled at fs Hz
    #  Inputs:
    #         bits  - bitarray of bits
    #         fs    - sampling rate
    # Outputs:
    #         sig    -  returns afsk1200 modulated signal
        # For you to complete   
        fss = lcm((1200,self.fs))
        deci = fss//self.fs

        Nb = fss//1200
        nb = len(bits)
        NRZ = ones((nb,Nb))
        for n in range(0,nb):
            if bits[n]:
                NRZ[n,:]=-NRZ[n,:]

        freq = 1700 + 500*NRZ.ravel()
        ph = 2.0*pi*integrate.cumtrapz(freq)/fss
        sig = cos(ph[::deci])
        
        return sig 
    
    
    def modulatePacket(self, callsign, digi, dest, info, preflags=80, postflags=80 ):
        
        # given callsign, digipath, dest, info, number of pre-flags and post-flags the function contructs
        # an appropriate aprs packet, then converts them to NRZI and calls `modulate` 
        # to afsk 1200 modulate the packet. 
        
        packet = ax25.UI(destination=dest,source=callsign, info=info, digipeaters=digi.split(b','),)
        prefix = barr.bitarray(np.tile([0,1,1,1,1,1,1,0],(preflags,)).tolist())
        suffix = barr.bitarray(np.tile([0,1,1,1,1,1,1,0],(postflags,)).tolist())
        sig = self.modulate(self.NRZ2NRZI(prefix + packet.unparse()+suffix))

        return sig
    

    def demod(self, buff):
        #Demodulates a buffer and returns valid NRZ
    
        # Similar to afsk1200_demod,  for you to complete
        sig = np.convolve(buff.copy(),self.h_bp,mode='valid')
        mark = abs(np.convolve(sig,self.h_mark,mode='valid'))
        space = abs(np.convolve(sig,self.h_space,mode='valid'))
        NRZ = mark-space
        NRZ = np.convolve(NRZ,self.h_lpp,'valid')
        
        return NRZ


    def FastPLL(self,NRZa):
        recbits = np.zeros(len(NRZa)//(self.fs//1200)*2,dtype=np.int32)
        pll = np.zeros(1,dtype = np.int32)
        pll[0] = self.pll
        ppll = np.zeros(1,dtype = np.int32)
        ppll[0] = self.ppll
        
        #print("pll = ",pll,"   ppll=",ppll)
        
        NRZb = (NRZa > 0).astype(np.int32)
        tot = libcd.pll(NRZb,len(NRZb),recbits,recbits,pll,ppll,self.dpll,self.plla)
        
        self.ppll = ppll.copy()
        self.pll = pll.copy()
        
        #print("post: pll = ",pll,"   ppll=",ppll)
        
        return barr.bitarray(recbits[:tot].tolist())
    
    def PLL(self, NRZa):
       #print("running PLL")
        idx = zeros(len(NRZa)//int(self.Ns)*2)   # allocate space to save indexes        
        c = 0
        
        for n in range(1,len(NRZa)):
            if (self.pll < 0) and (self.ppll >0):
                idx[c] = n
                c = c+1
        
            if (NRZa[n] >= 0) !=  (NRZa[n-1] >=0):
                self.pll = np.int32(self.pll*self.plla)    
        
            self.ppll = self.pll
            self.pll = np.int32(self.pll+ self.dpll)
    
        return idx[:c].astype(np.int32) 
    

    def findPackets(self,bits):
        # function take a bitarray and looks for AX.25 packets in it. 
        # It implements a 2-state machine of searching for flag or collecting packets
        flg = barr.bitarray([0,1,1,1,1,1,1,0])
        packets = []
        n = self.bitpointer
        
        # Loop over bits
        while (n < len(bits)-7) :
            # default state is searching for packets
            if self.state is 'search':
                # look for 1111110, because can't be sure if the first zero is decoded
                # well if the packet is not padded.
                if bits[n:n+7] == flg[1:]:
                    # flag detected, so switch state to collecting bits in a packet
                    # start by copying the flag to the packet
                    # start counter to count the number of bits in the packet
                    self.state = 'pkt'
                    self.packet=flg.copy()
                    self.pktcounter = 8
                    # Advance to the end of the flag
                    n = n + 7
                else:
                    # flag was not found, advance by 1
                    n = n + 1             
            # state is to collect packet data. 
            elif self.state is 'pkt':
                # Check if we reached a flag by comparing with 0111111
                # 6 times ones is not allowed in a packet, hence it must be a flag (if there's no error)
                if bits[n:n+7] == flg[:7]:
                    # Flag detected, check if packet is longer than some minimum
                    if self.pktcounter > 200:
                        #print('packet found!')
                        # End of packet reached! append packet to list and switch to searching state
                        # We don't advance pointer since this our packet might have been
                        # flase detection and this flag could be the beginning of a real packet
                        self.state = 'search'
                        self.packet.extend(flg)
                        packets.append(self.packet.copy())
                    else:
                        # packet is too short! false alarm. Keep searching 
                        # We don't advance pointer since this this flag could be the beginning of a real packet
                        self.state = 'search'
                # No flag, so collect the bit and add to the packet
                else:
                    # check if packet is too long... if so, must be false alarm
                    if self.pktcounter < 2680:
                        # Not a false alarm, collect the bit and advance pointer        
                        self.packet.append(bits[n])
                        self.pktcounter = self.pktcounter + 1
                        n = n + 1
                    else:  #runaway packet
                        #runaway packet, switch state to searching, and advance pointer
                        self.state = 'search'
                        n = n + 1
        
        self.bitpointer = n-(len(bits)-7) 
        return packets

    
    # function to generate a checksum for validating packets
    def genfcs(self,bits):
        # Generates a checksum from packet bits
        fcs = ax25.FCS()
        for bit in bits:
            fcs.update_bit(bit)
    
        digest = barr.bitarray(endian="little")
        digest.frombytes(fcs.digest())

        return digest


    # function to parse packet bits to information
    def decodeAX25(self, bits):
        ax = ax25.AX25()
        ax.info = "bad packet"
        ax.parse(bits)
        return ax
    
    
    def processBuffer(self, buff_in):
        
        # function processes an audio buffer. It collect several small into a large one
        # Then it demodulates and finds packets.
        #
        # The function uses overlap and save convolution
        # The function returns packets when they become available. Otherwise, returns empty list
        
        N = self.N
        NN = (N*3 -3 )
        
        Nchunks = self.Nchunks
        Abuffer = self.Abuffer
        fs = self.fs
        Ns = self.Ns
        
        validPackets=[]
        packets=[]
        NRZI=[]
        idx = []
        bits = []
        
        # Fill in buffer at the right place
        self.buff[NN+self.chunk_count*Abuffer:NN+(self.chunk_count+1)*Abuffer] = buff_in.copy()
        self.chunk_count = self.chunk_count + 1  
        
        # number of chunk reached -- process large buffer
        if self.chunk_count == Nchunks:
            # Demodulate to get NRZI
            NRZI = self.demod(self.buff)
            # compute sampling points, using PLL
            #idx = self.PLL(NRZI)
            # Sample and make a decision based on threshold
            #bits = barr.bitarray((NRZI[idx]>0).tolist())
            
            bits = self.FastPLL(NRZI)
            # In case that buffer is too small raise an error -- must have at least 7 bits worth
            if len(bits) < 7:
                raise ValueError('number of bits too small for buffer')
            
            # concatenate end of previous buffer to current one
            bits = self.oldbits + self.NRZI2NRZ(bits)
            
            # store end of bit buffer to next buffer
            self.oldbits = bits[-7:].copy()
            
            # look for packets
            packets = self.findPackets(bits)
            
            # Copy end of sample buffer to the beginning of the next (overlapp and save)
            self.buff[:NN] = self.buff[-NN:].copy()
            
            # reset chunk counter
            self.chunk_count = 0
            
            # checksum test for all detected packets
            for n in range(0,len(packets)):
                if len(packets[n]) > 200: 
                    try:
                        ax = self.decodeAX25(packets[n])
                    except:
                        ax = ax25.AX25()
                        ax.info = "bad packet"
                    if ax.info != 'bad packet' and ax.info != 'no decode':
                        validPackets.append(packets[n])                
  
        return validPackets

B64/bitarray and enqueueing

In [37]:
# Convert from bitarray class to b64 bytes and back
def ee123_bitarr_to_base64(bits: barr.bitarray):
    bN = np.uint32(len(bits)).tobytes()
    Bytes = bN + bits.tobytes()
    return base64.b64encode(Bytes)

def ee123_base64_to_bitarr(b64: str):
    Bytes = base64.b64decode(b64)  # Be careful not to overwrite the builtin bytes class!
    N = np.frombuffer(Bytes[:4], dtype='<u4')[0]
    ba = barr.bitarray()
    ba.frombytes(Bytes[4:])
    return ba[:N]

def file_to_b64(fname):
    with open(fname, 'rb') as f:
        raw = f.read()
    # Prepend the number of bytes before encoding
    b64 = base64.b64encode(np.uint32(len(raw)).tobytes() + raw)
    return b64

def enqueue_data(callsign, modem, data, address=None, uid=None, fname="myfile.bin", 
                 comment="UCB EE123 message format", dest=b"APCAL", bsize=240):
    """
    Inputs:
      callsign: your callsign
      modem: a modem object
      data: data to send as a bytes object
      address: address callsign for the message, defaults to your callsign
      uid: optionally specify a UID, otherwise one will be randomly chosen
      fname: filename to send in start packet
      comment: comment for start packet
      dest: should be APCAL for EE123 data
      bsize: number of bytes to send per packet
      
    Outputs:
      Qout: a queue containing modulated packets to transmit
      uid: the uid used for this data transfer
    """
    if address is None:
        address = callsign
    if uid is None:
        uid = np.random.randint(0, 10000)
    uid = bytes("{:04d}".format(uid), 'utf-8')
    print("Putting packets in Queue for transmission ID=%s" % uid.decode())
    
    Qout = Queue.Queue()
    
    # For you to complete
    Digi =b'WIDE1-1,WIDE2-1'
    #TGTCALL is the callsign of the addressee
    #YOURCALL is your own callsign
    pad_callsign = callsign + " "*(9-len(callsign))
    pad_addr = address + " "*(9-len(address))
    # start packet
    info = b":" + bytes(pad_addr, 'utf-8') + b":START" + uid + b"," + bytes(fname, 'utf-8') + b"," + bytes(comment, 'utf-8')
    sig_start = modem.modulatePacket(callsign, Digi, dest, info)
    Qout.put(sig_start)
    
    #data packets
    #data = data[4:] #there's a uint32 (4B) at the start that records number of bytes, and it's messing up the decoding
    seq_num = 0
    for i in range(0,len(data),bsize):
        data_blk = data[i:i+bsize]
        seq_bytes = bytes("{:04d}".format(seq_num), 'utf-8')
        info = b":" + bytes(pad_addr, 'utf-8') + b":" + data_blk + seq_bytes
        sig = modem.modulatePacket(callsign, Digi, dest, info)
        seq_num += 1
        Qout.put(sig)
    #end packet
    info = b":" + bytes(pad_addr, 'utf-8') + b":END" + uid
    sig_end = modem.modulatePacket(callsign, Digi, dest, info)
    Qout.put(sig_end)
    return Qout, uid

callback

In [17]:
# Define the callback factory so we can specify which non-global queue we want
def make_QR_callback(Qin):
    def queueREPLAY_callback(indata, outdata, frames, time, status):
        assert frames == 1024
        if status:
            print(status)
        outdata[:] = indata
        Qin.put(indata.copy()[:, 0]) # Global queue
    return queueREPLAY_callback

decode

In [20]:
def decode_ee123_message(msg):
    """
    Input:
      msg: The info field of an APRS message packet
    
    Output:
      A dict containing the key components the packet may contain
      
    Message formats:
    :CALLSIGN :START####,filename,comment
    :CALLSIGN :<data data data>####
    :CALLSIGN :END####
    """
    msg = msg.strip()
    addressee = ''
    uid = None
    isstart = False
    isend = False
    seq = None
    data = None  # This should be in base64
    filename = ''
    comment = ''
    
    # Your code here
    fields = msg.split(':')
    addressee = fields[1].strip()
    data_fields = fields[2].strip().split(',')
    isstart = 'START' in data_fields[0]
    isend = 'END' in data_fields[0]
    if isstart:
        uid = data_fields[0].strip()[-4:]
        filename = data_fields[1].strip()
        comment = data_fields[2].strip()
    elif isend:
        uid = data_fields[0].strip()[-4:]
    else:
        seq = data_fields[0][-4:]
        data = data_fields[0][:-4]
    
    return {'addr': addressee, 'uid': uid, 'isstart': isstart, 
            'isend': isend, 'seq': seq, 'filename': filename, 
            'data': data, 'comment': comment}

In [21]:
def decode_from_queue(modem, Qin)
    # Decode the recording
    print("decoding packets")
    starttime = time.time()

    # For you to complete
    # Store the name of the reconstructed file in a variable called rec_fname
    packets = []
    while not Qin.empty():
        packets.extend(modem.processBuffer(Qin.get()))
    rec_uid_start = None
    rec_uid_end = None
    rec_fname = ""
    rec_data = None
    #print(len(packets))
    for pkt in packets:
        #print(len(pkt))
        #print("decoding individual packet")
        ax_msg = modem.decodeAX25(pkt)
        #print(ax_msg)
        if ax_msg.info != 'bad packet':
            ax_info = decode_ee123_message(ax_msg.info)
            if ax_info['isstart']:
                rec_uid_start = ax_info['uid']
                rec_fname = ax_info['filename']
            elif ax_info['isend']:
                rec_uid_end = ax_info['uid']
                break
            else:
                if rec_data == None:
                    rec_data = ax_info['data']
                else:
                    rec_data = rec_data + ax_info['data']
        else:
            print("bad packet")
    print(rec_uid_start)
    print(rec_uid_end)
    print(rec_fname)
    rec_data = base64.b64decode(rec_data)
    rec_data_len = np.frombuffer(rec_data[:4], dtype='<u4')[0]
    rec_data = rec_data[4:rec_data_len]
    print("Done: {:.2f}s".format(time.time() - starttime))
    return rec_data #array of bytes

SyntaxError: invalid syntax (<ipython-input-21-f2c7b445f3b5>, line 1)

## Final Stuff

In [30]:
#Load image
filename = "Rolling.png"
im_stack = hfunc.imageStack_load(filename)

In [40]:
#create folder
path = filename[:filename.find('.')]+"_encoded"+'/'
os.system("rm -rf {:s}".format(path))
os.system("mkdir {:s}".format(path))

print(im_stack.shape)
frame_ctr = 0
for im in im_stack:
    outname = path+"frame_"+"{:02d}".format(frame_ctr)+".jpeg123"
    jpeg123_encoder(im, outname)
    frame_ctr += 1

(13, 42, 52, 3)


In [42]:
print(frame_ctr)

13


In [44]:
#decode jpeg123 and convert to tiff
dec_stack = []
#decode jpeg123
for i in range(frame_ctr):
    outname = path+"frame_"+"{:02d}".format(i)+".jpeg123"
    dec_stack.append(jpeg123_decoder(outname))
#save as tiff
#path for decoded tiff images
path_dec = filename[:filename.find('.')]+"_decoded"+'/'
os.system("rm -rf {:s}".format(path_dec))
os.system("mkdir {:s}".format(path_dec))
framerate=12 #given on ee123 website
for i in range(len(dec_stack)):
    Image.fromarray(dec_stack[i]).save('%s/frame_%02d.tiff' % (path_dec, i))
hfunc.GIF_save(path_dec, framerate)

In [46]:
#psnr
print("PSNR:")
print(hfunc.psnr(im_stack, np.array(dec_stack)))

PSNR:
19.825781919292325


In [51]:
#using tiff_play on the decoded stuff
hfunc.Tiff_play(path_dec+"*.tiff", 1000, framerate)

In [98]:
#turn jpeg123 into packets
callsign = "KN6IDV-3"
fs = 48000
modem = TNCaprs(fs = fs ,Abuffer = 1024,Nchunks = 1)
addr="KN6IDV"
Qout = None
uid = None
vid_data = None
for i in range(frame_ctr):
    #concatenating b64 strings together doesn't work nicely
    #frm_bytes = file_to_b64('%s/frame_%02d.jpeg123' % (path, i))
    #modified from file_to_b64
    with open('%s/frame_%02d.jpeg123' % (path, i), 'rb') as f:
        raw = f.read()
    data = np.uint32(len(raw)).tobytes() + raw
    if vid_data == None:
        vid_data = data
    else:
        vid_data += data
    vid_b64 = base64.b64encode(vid_data)
Qout, uid = enqueue_data(callsign, modem, vid_b64, addr, fname=filename[:filename.find('.')]+'.jpeg123')
print("There are %d packets in the queue" %Qout.qsize())

Putting packets in Queue for transmission ID=1976
There are 104 packets in the queue


In [99]:
#send and receive on loopback
#DO NOT run Direwolf!
Qin = Queue.Queue()
# This stream captures samples on loopback_rcv and plays them on the built in device.
# Remember that loopback is connected to loopback_rcv.
st_replay = sd.Stream( device=(loop_idx_rcv, builtin_idx), blocksize=modem.Abuffer, callback=make_QR_callback(Qin))
print("Starting streams")
st_replay.start()
starttime = time.time()

# For you to complete
while not Qout.empty():
    msg = Qout.get()
    sd.play(msg*0.2, samplerate=48000, device=loop_idx, blocking=True)
    #time.sleep(4) #when transmitting over Direwolf

# Remember, we have to stop and close the stream.
st_replay.stop()
st_replay.close()

print("Done")
print(time.time() - starttime)

Starting streams
Done
311.1348340511322


In [110]:
#create folder to receive
#TODO: figure out a way to make folder based on received files?
path_enc_rcv = "received_stack/" #fix this: currently makes path FILENAME_encoded/_trans/
os.system("rm -rf {:s}".format(path_enc_rcv))
os.system("mkdir {:s}".format(path_enc_rcv))

#decode received packets
print("dequeuing packets")
starttime = time.time()
tgt_addr = "KN6IDV" #change to whatever callsign should be receiving the packets, no ssid
# For you to complete
# Store the name of the reconstructed file in a variable called rec_fname
# packets = []
# while not Qin.empty():
#     packets.extend(modem.processBuffer(Qin.get()))
# print("Done dequeuing: {:.2f}s".format(time.time() - starttime))
#comment/uncomment the 4 lines above when running multiple times/fresh
rec_uid_start = None
rec_uid_end = None
rec_fname = ""
rec_data = None
print("decoding packets")
rec_data_arr = []
for pkt in packets:
    ax_msg = modem.decodeAX25(pkt)
    #print(ax_msg)
    if ax_msg.info != 'bad packet':
        ax_info = decode_ee123_message(ax_msg.info)
        if ax_info['addr'] == tgt_addr:
            if ax_info['isstart']:
                rec_uid_start = ax_info['uid']
                rec_fname = ax_info['filename']
                print("start transmission", rec_uid_start)
            elif ax_info['isend']:
                rec_uid_end = ax_info['uid']
                print("end transmission", rec_uid_end)
            else:
                print("data received")
                rec_data_arr.append((ax_info['seq'], ax_info['data']))
    else:
        print("bad packet")

#sort the received data packets
rec_data_arr.sort(key=lambda x: x[0])
for _, info in rec_data_arr:
    if rec_data == None:
        rec_data = info
    else:
        rec_data += info
print(rec_fname)
        
print("Done: {:.2f}s".format(time.time() - starttime))

dequeuing packets
decoding packets
start transmission 1976
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
data received
dat

In [114]:
rec_data_dec = base64.b64decode(rec_data)
# rec_data_len = np.frombuffer(rec_data_dec[:4], dtype='<u4')[0]
# print(rec_data_len)
# print(type(rec_data_len))
for i in range(frame_ctr):
    print("%d bytes remaining" % len(rec_data_dec))
    rec_data_len = np.frombuffer(rec_data_dec[:4], dtype='<u4')[0]
    print("writing %d bytes..." % rec_data_len)

    rec_data_dec_solo = rec_data_dec[4:rec_data_len+4]
    with open(path_enc_rcv + "frame_%02d.jpeg123" % i, 'wb+') as file:
        file.write(rec_data_dec_solo)
    rec_data_dec = rec_data_dec[rec_data_len+4:]
    print("frame %02d %d bytes written" % (i, len(rec_data_dec_solo)))

18186 bytes remaining
writing 1380 bytes...
frame 00 1380 bytes written
16802 bytes remaining
writing 1406 bytes...
frame 01 1406 bytes written
15392 bytes remaining
writing 1388 bytes...
frame 02 1388 bytes written
14000 bytes remaining
writing 1376 bytes...
frame 03 1376 bytes written
12620 bytes remaining
writing 1372 bytes...
frame 04 1372 bytes written
11244 bytes remaining
writing 1402 bytes...
frame 05 1402 bytes written
9838 bytes remaining
writing 1394 bytes...
frame 06 1394 bytes written
8440 bytes remaining
writing 1442 bytes...
frame 07 1442 bytes written
6994 bytes remaining
writing 1402 bytes...
frame 08 1402 bytes written
5588 bytes remaining
writing 1426 bytes...
frame 09 1426 bytes written
4158 bytes remaining
writing 1390 bytes...
frame 10 1390 bytes written
2764 bytes remaining
writing 1362 bytes...
frame 11 1362 bytes written
1398 bytes remaining
writing 1394 bytes...
frame 12 1394 bytes written


In [116]:
#decode transmitted video
dec_stack = []
#decode jpeg123
for i in range(frame_ctr):
    outname = path_enc_rcv+"frame_{:02d}.jpeg123".format(i)
    dec_stack.append(jpeg123_decoder(outname))
#save as tiff
#path for decoded tiff images
path_dec_trans = path_enc_rcv+"_dec/"
os.system("rm -rf {:s}".format(path_dec_trans))
os.system("mkdir {:s}".format(path_dec_trans))
framerate=12 #given on ee123 website
#TODO: transfer framerate in packets too
for i in range(len(dec_stack)):
    Image.fromarray(dec_stack[i]).save('%s/frame_%02d.tiff' % (path_dec_trans, i))
hfunc.GIF_save(path_dec_trans, framerate)

In [117]:
#psnr of received video
print("PSNR:")
print(hfunc.psnr(im_stack, np.array(dec_stack)))

PSNR:
19.825781919292325


In [118]:
#play decoded transmitted video
hfunc.Tiff_play(path_dec+"*.tiff", 800, framerate)